# preamble

In [1]:
# !pip install -r requirements.txt -q

In [2]:
import requests
import os
import openai

from dotenv import load_dotenv
load_dotenv();

In [4]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
NOTION_API_KEY = os.environ["NOTION_API_KEY"]
NOTION_DATABASE_ID = os.environ["NOTION_DATABASE_ID"]

# receive voicenote

In [ ]:
# todo

# transcribe

In [3]:
voice_note = open("./voicenotes/signal-2023-08-12-133923.mp3", "rb")
transcript = openai.Audio.transcribe(api_key=OPENAI_API_KEY,
                                     model="whisper-1",
                                     file=voice_note,
                                     response_format="text",
                                     language="en"
                                     )

# transcript

In [21]:
transcript

"And then it's gone. I think it's a bit like what you just said, the creative process falls away. Then you have to ask one question deeper, what do you want with your education? What do you want that diploma to say? What do you want to express? I think you want people to be able to bring innovation. And I think to do that, you first have to get people to do assignments and things, which is an answer to that. I also start with experiments where it is clear how it should go. If you completely rely on things like that, on things that are already known, then those things could also be done with the help of AI. But I think people take less other lessons with them, if you put it like that. It's just difficult. I think in the scenario that you have a person who looks very critically at the output of JCPT, who has sufficient knowledge about the subject, that they can make judgments. And who sometimes does things on their own. Then I think it's fine. But I'm not sure how to do that. But what is

### split long files

In [11]:
# not needed when also 16k context model can be used

# max_tokens = 4097
# avg_token_len = 4
# max_trans_len = max_tokens * avg_token_len - 100 #for safety

# len(transcript) / max_trans_len

2.4601547151277012

# generate summary, topic list, and critical questions

In [4]:
# summary_prompt      = """\
# You are a helpful assistant, specialized in summarizing and understanding the core of a given message. \
# For the given piece of text, you are going to perform the following four tasks. \
# Firstly, start your response with a consice title describing the text. \
# Secondly, summarize in first person the content of this text in about one alinea. \
# Thirdly, list the different topics discussed in this text. Only add a topic when it is discussed for at least a couple of sentences. \
# Fourth, imagine being a critical life coach of the person writing this text. What are the three best questions you could ask regarding this text?\
# """

In [22]:
summary_prompt      = """\
You are specialized in summarizing and understanding the core of a given message. \
For the given piece of text, perform the following four tasks. \
First, provide a concise title describing the text. \
Second, summarize in first person the content of this text. \
Third, list the different topics discussed in this text. \
Fourth, imagine being a critical life coach. Provide the three best questions you could ask regarding this text.\
"""

In [24]:
text_response = openai.ChatCompletion.create(api_key=OPENAI_API_KEY,
                                             model="gpt-3.5-turbo",
                                             messages=[
                                                 {"role": "system", "content": summary_prompt},
                                                 {"role": "user", "content": transcript}
                                             ])

In [25]:
text_response["choices"][0]["message"]["content"]

'Title: The Future of Education, AI, and Democracy\n\nSummary: The conversation touches on various topics, including the role of AI in education, the importance of independent thinking and experimentation, the limitations and biases of AI, the need for critical thinking skills, the balance between using technology and mastering fundamental skills, the potential impact of AI on society, the challenges of democracy, and the quest for balance and meaning in life.\n\nTopics discussed:\n1. The role of AI in education\n2. The importance of independent thinking and experimentation\n3. The limitations and biases of AI\n4. The balance between using technology and mastering fundamental skills\n5. The potential impact of AI on society\n6. The challenges of democracy\n7. The quest for balance and meaning in life\n\nThree questions for a critical life coach:\n1. How can we foster independent thinking and experimentation in education while also utilizing AI technologies?\n2. How can we address the l

## process response

In [26]:
content_notion = text_response["choices"][0]["message"]["content"].split("\n\n")

In [27]:
title_notion = 'Not found'
summary_notion = 'Not found'
topics_notion = 'Not found'
critical_questions_notion = 'Not found'

title_notion = content_notion[0]
summary_notion = content_notion[1]
topics_notion = content_notion[2]
critical_questions_notion = content_notion[3]

# create Notion page

In [28]:
url = "https://api.notion.com/v1/pages/"

headers = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Notion-Version": "2022-06-28",
    "Content-Type": "application/json"
}

data_database = {

    "parent":   {
        "type": "database_id",
        "database_id": NOTION_DATABASE_ID
                },
    "icon": {"emoji": "🎙️"},

    "properties":   {
        "Name":         {
            "title": [{"text": {"content": title_notion}}]
                        }  
                    },

    "children": [
        {
            "object": "block",
            "heading_2": {"rich_text": [{"text": {"content": "Summary"}}]}
        },
        {
            "object": "block",
            "paragraph": {"rich_text": [{"text": {"content": summary_notion}}], "color": "default"}
        },
        {
            "object": "block",
            "heading_2": {"rich_text": [{"text": {"content": "Topics"}}]}
        },
        {
            "object": "block",
            "paragraph": {"rich_text": [{"text": {"content": topics_notion}}], "color": "default"}
        },
        {
            "object": "block",
            "heading_2": {"rich_text": [{"text": {"content": "Critical Questions"}}]}
        },
        {
            "object": "block",
            "paragraph": {"rich_text": [{"text": {"content": critical_questions_notion}}], "color": "default"}
        },
        # {
        #     "object": "block",
        #     "heading_2": {"rich_text": [{"text": {"content": "Transcript"}}]}
        # },
        # {
        #     "object": "block",
        #     "paragraph": {"rich_text": [{"text": {"content": transcript}}], "color": "default"}
        # },
                ]

}

response_get = requests.post(url, headers=headers, json=data_database)
response_get.json()

{'object': 'page',
 'id': 'd2216952-8b29-4361-be0a-9fc1d372bb17',
 'created_time': '2023-08-23T10:53:00.000Z',
 'last_edited_time': '2023-08-23T10:53:00.000Z',
 'created_by': {'object': 'user',
  'id': 'dc664599-409b-4fbc-b638-fa34c238bb19'},
 'last_edited_by': {'object': 'user',
  'id': 'dc664599-409b-4fbc-b638-fa34c238bb19'},
 'cover': None,
 'icon': {'type': 'emoji', 'emoji': '🎙️'},
 'parent': {'type': 'database_id',
  'database_id': 'e5568bc0-5015-4ec7-bd8c-ba5050353de6'},
 'archived': False,
 'properties': {'Created time': {'id': 'BAiK',
   'type': 'created_time',
   'created_time': '2023-08-23T10:53:00.000Z'},
  'Name': {'id': 'title',
   'type': 'title',
   'title': [{'type': 'text',
     'text': {'content': 'Title: The Future of Education, AI, and Democracy',
      'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': 'Title: The Future 